In [140]:

import math
import torch
import torch.nn as nn

In [141]:
gt_mask = torch.ones((2, 4))
gt_mask[0, 3] = 0
gt_mask = gt_mask.unsqueeze(-1)
gt_mask = gt_mask.expand(-1, -1, 8).bool()
print(gt_mask)

tensor([[[ True,  True,  True,  True,  True,  True,  True,  True],
         [ True,  True,  True,  True,  True,  True,  True,  True],
         [ True,  True,  True,  True,  True,  True,  True,  True],
         [False, False, False, False, False, False, False, False]],

        [[ True,  True,  True,  True,  True,  True,  True,  True],
         [ True,  True,  True,  True,  True,  True,  True,  True],
         [ True,  True,  True,  True,  True,  True,  True,  True],
         [ True,  True,  True,  True,  True,  True,  True,  True]]])


In [142]:
align_metric=torch.tensor([[[10,1,2,1,1,1,1,10],
                            [3,4,5,1,1,1,1,1],
                            [7,8,9,1,1,1,1,1],
                            [1,4,7,1,1,1,1,1]],

                            [[10,1,2,1,1,1,1,10],
                            [0,4,5,1,1,1,1,1],
                            [7,0,0,1,1,9,1,1],
                            [1,4,7,1,1,1,1,1]]],dtype=torch.float32)

topk = torch.topk(align_metric, k=3, dim=-1, largest=True)
print(align_metric)
print(topk[0])
print(topk[1])

tensor([[[10.,  1.,  2.,  1.,  1.,  1.,  1., 10.],
         [ 3.,  4.,  5.,  1.,  1.,  1.,  1.,  1.],
         [ 7.,  8.,  9.,  1.,  1.,  1.,  1.,  1.],
         [ 1.,  4.,  7.,  1.,  1.,  1.,  1.,  1.]],

        [[10.,  1.,  2.,  1.,  1.,  1.,  1., 10.],
         [ 0.,  4.,  5.,  1.,  1.,  1.,  1.,  1.],
         [ 7.,  0.,  0.,  1.,  1.,  9.,  1.,  1.],
         [ 1.,  4.,  7.,  1.,  1.,  1.,  1.,  1.]]])
tensor([[[10., 10.,  2.],
         [ 5.,  4.,  3.],
         [ 9.,  8.,  7.],
         [ 7.,  4.,  1.]],

        [[10., 10.,  2.],
         [ 5.,  4.,  1.],
         [ 9.,  7.,  1.],
         [ 7.,  4.,  1.]]])
tensor([[[7, 0, 2],
         [2, 1, 0],
         [2, 1, 0],
         [2, 1, 7]],

        [[7, 0, 2],
         [2, 1, 7],
         [5, 0, 6],
         [2, 1, 7]]])


In [143]:
topk_mask = torch.ones((2, 4, 1))
topk_mask[0, 3] = 0
topk_mask = topk_mask.expand(-1, -1, 3).bool()
print(topk_mask)

topk_idxs = topk[1]
topk_idxs.masked_fill_(~topk_mask, 0)
print(topk_idxs)



tensor([[[ True,  True,  True],
         [ True,  True,  True],
         [ True,  True,  True],
         [False, False, False]],

        [[ True,  True,  True],
         [ True,  True,  True],
         [ True,  True,  True],
         [ True,  True,  True]]])
tensor([[[7, 0, 2],
         [2, 1, 0],
         [2, 1, 0],
         [0, 0, 0]],

        [[7, 0, 2],
         [2, 1, 7],
         [5, 0, 6],
         [2, 1, 7]]])


In [144]:
count_tensor = torch.zeros(align_metric.shape, dtype=torch.int8, device=topk_idxs.device)
ones = torch.ones_like(topk_idxs[:, :, :1], dtype=torch.int8, device=topk_idxs.device)
for k in range(3):
    count_tensor.scatter_add_(-1, topk_idxs[:, :, k:k + 1], ones)
count_tensor.masked_fill_(count_tensor > 1, 0)
print(count_tensor)

tensor([[[1, 0, 1, 0, 0, 0, 0, 1],
         [1, 1, 1, 0, 0, 0, 0, 0],
         [1, 1, 1, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0]],

        [[1, 0, 1, 0, 0, 0, 0, 1],
         [0, 1, 1, 0, 0, 0, 0, 1],
         [1, 0, 0, 0, 0, 1, 1, 0],
         [0, 1, 1, 0, 0, 0, 0, 1]]], dtype=torch.int8)


In [145]:
mask_pos = gt_mask * count_tensor
mask_pos

tensor([[[1, 0, 1, 0, 0, 0, 0, 1],
         [1, 1, 1, 0, 0, 0, 0, 0],
         [1, 1, 1, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0]],

        [[1, 0, 1, 0, 0, 0, 0, 1],
         [0, 1, 1, 0, 0, 0, 0, 1],
         [1, 0, 0, 0, 0, 1, 1, 0],
         [0, 1, 1, 0, 0, 0, 0, 1]]], dtype=torch.int8)

In [146]:
fg_mask = mask_pos.sum(-2)
fg_mask

tensor([[3, 2, 3, 0, 0, 0, 0, 1],
        [2, 2, 3, 0, 0, 1, 1, 3]])

In [147]:
mask_multi_gts = (fg_mask.unsqueeze(1) > 1).expand(-1, 4, -1)
mask_multi_gts

tensor([[[ True,  True,  True, False, False, False, False, False],
         [ True,  True,  True, False, False, False, False, False],
         [ True,  True,  True, False, False, False, False, False],
         [ True,  True,  True, False, False, False, False, False]],

        [[ True,  True,  True, False, False, False, False,  True],
         [ True,  True,  True, False, False, False, False,  True],
         [ True,  True,  True, False, False, False, False,  True],
         [ True,  True,  True, False, False, False, False,  True]]])

In [148]:
overlaps = torch.rand((2, 4, 8))
overlaps

tensor([[[0.0148, 0.5420, 0.0744, 0.3701, 0.5614, 0.0080, 0.3767, 0.3373],
         [0.6202, 0.2900, 0.7835, 0.6297, 0.6616, 0.9272, 0.0851, 0.2147],
         [0.9434, 0.7546, 0.3585, 0.0242, 0.7593, 0.7786, 0.5190, 0.8372],
         [0.3798, 0.2589, 0.8041, 0.1986, 0.2092, 0.5301, 0.3496, 0.0403]],

        [[0.1429, 0.8005, 0.4486, 0.1503, 0.9951, 0.3046, 0.9802, 0.1799],
         [0.0230, 0.8320, 0.6076, 0.5870, 0.9751, 0.8765, 0.4471, 0.6931],
         [0.4278, 0.2421, 0.9587, 0.6602, 0.2421, 0.5515, 0.8688, 0.9818],
         [0.1962, 0.5363, 0.4034, 0.0046, 0.3641, 0.0843, 0.4413, 0.8637]]])

In [149]:
max_overlaps_idx = overlaps.argmax(1)  # (b, h*w)
max_overlaps_idx

tensor([[2, 2, 3, 1, 2, 1, 2, 2],
        [2, 1, 2, 2, 0, 1, 0, 2]])

In [150]:
is_max_overlaps = torch.zeros(mask_pos.shape, dtype=mask_pos.dtype, device=mask_pos.device)
is_max_overlaps

tensor([[[0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0]],

        [[0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0]]], dtype=torch.int8)

In [151]:
max_overlaps_idx.unsqueeze(1)

tensor([[[2, 2, 3, 1, 2, 1, 2, 2]],

        [[2, 1, 2, 2, 0, 1, 0, 2]]])

In [152]:
is_max_overlaps.scatter_(1, max_overlaps_idx.unsqueeze(1), 1)

tensor([[[0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 1, 0, 1, 0, 0],
         [1, 1, 0, 0, 1, 0, 1, 1],
         [0, 0, 1, 0, 0, 0, 0, 0]],

        [[0, 0, 0, 0, 1, 0, 1, 0],
         [0, 1, 0, 0, 0, 1, 0, 0],
         [1, 0, 1, 1, 0, 0, 0, 1],
         [0, 0, 0, 0, 0, 0, 0, 0]]], dtype=torch.int8)

In [153]:
mask_multi_gts

tensor([[[ True,  True,  True, False, False, False, False, False],
         [ True,  True,  True, False, False, False, False, False],
         [ True,  True,  True, False, False, False, False, False],
         [ True,  True,  True, False, False, False, False, False]],

        [[ True,  True,  True, False, False, False, False,  True],
         [ True,  True,  True, False, False, False, False,  True],
         [ True,  True,  True, False, False, False, False,  True],
         [ True,  True,  True, False, False, False, False,  True]]])

In [154]:
mask_pos

tensor([[[1, 0, 1, 0, 0, 0, 0, 1],
         [1, 1, 1, 0, 0, 0, 0, 0],
         [1, 1, 1, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0]],

        [[1, 0, 1, 0, 0, 0, 0, 1],
         [0, 1, 1, 0, 0, 0, 0, 1],
         [1, 0, 0, 0, 0, 1, 1, 0],
         [0, 1, 1, 0, 0, 0, 0, 1]]], dtype=torch.int8)

In [155]:
mask_pos = torch.where(mask_multi_gts, is_max_overlaps, mask_pos).float()
mask_pos

tensor([[[0., 0., 0., 0., 0., 0., 0., 1.],
         [0., 0., 0., 0., 0., 0., 0., 0.],
         [1., 1., 0., 0., 0., 0., 0., 0.],
         [0., 0., 1., 0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 1., 0., 0., 0., 0., 0., 0.],
         [1., 0., 1., 0., 0., 1., 1., 1.],
         [0., 0., 0., 0., 0., 0., 0., 0.]]])

In [156]:
fg_mask = mask_pos.sum(-2)
fg_mask

tensor([[1., 1., 1., 0., 0., 0., 0., 1.],
        [1., 1., 1., 0., 0., 1., 1., 1.]])

In [157]:
target_gt_idx = mask_pos.argmax(-2)
target_gt_idx

tensor([[2, 2, 3, 0, 0, 0, 0, 0],
        [2, 1, 2, 0, 0, 2, 2, 2]])

In [158]:
gt_labels = torch.randint(0, 16, (2, 4, 1))
gt_labels[0, 3, 0] = 0
gt_labels

tensor([[[ 9],
         [ 5],
         [ 1],
         [ 0]],

        [[ 7],
         [10],
         [ 0],
         [11]]])

In [159]:
# Assigned target labels, (b, 1)
batch_ind = torch.arange(end=2, dtype=torch.int64, device=gt_labels.device)[..., None]
batch_ind

tensor([[0],
        [1]])

In [160]:
target_gt_idx = target_gt_idx + batch_ind * 4
target_gt_idx

tensor([[2, 2, 3, 0, 0, 0, 0, 0],
        [6, 5, 6, 4, 4, 6, 6, 6]])

In [161]:
gt_labels.long().flatten()

tensor([ 9,  5,  1,  0,  7, 10,  0, 11])

In [162]:
target_labels = gt_labels.long().flatten()[target_gt_idx]
target_labels

tensor([[ 1,  1,  0,  9,  9,  9,  9,  9],
        [ 0, 10,  0,  7,  7,  0,  0,  0]])

In [163]:
gt_bboxes = torch.randint(0, 320, (2, 4, 4))
gt_bboxes

tensor([[[ 30, 289, 294,  41],
         [216, 170,  15,  90],
         [ 59,  10,  31, 115],
         [257, 271, 317, 271]],

        [[101, 261,  72,   6],
         [ 97,  71,  57,  78],
         [224,  44,  76, 189],
         [304, 215, 244,  58]]])

In [164]:
target_bboxes = gt_bboxes.view(-1, 4)[target_gt_idx]
target_bboxes

tensor([[[ 59,  10,  31, 115],
         [ 59,  10,  31, 115],
         [257, 271, 317, 271],
         [ 30, 289, 294,  41],
         [ 30, 289, 294,  41],
         [ 30, 289, 294,  41],
         [ 30, 289, 294,  41],
         [ 30, 289, 294,  41]],

        [[224,  44,  76, 189],
         [ 97,  71,  57,  78],
         [224,  44,  76, 189],
         [101, 261,  72,   6],
         [101, 261,  72,   6],
         [224,  44,  76, 189],
         [224,  44,  76, 189],
         [224,  44,  76, 189]]])

In [165]:
target_labels.clamp_(0)
target_labels

tensor([[ 1,  1,  0,  9,  9,  9,  9,  9],
        [ 0, 10,  0,  7,  7,  0,  0,  0]])

In [166]:
# 10x faster than F.one_hot()
target_scores = torch.zeros((target_labels.shape[0], target_labels.shape[1], 16),
                            dtype=torch.int64,
                            device=target_labels.device)  # (b, h*w, 80)

target_scores

tensor([[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],

        [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]])

In [167]:
target_scores.scatter_(2, target_labels.unsqueeze(-1), 1)
target_scores

tensor([[[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
         [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
         [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]],

        [[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
         [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
         [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
         [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
         [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]])

In [168]:
fg_scores_mask = fg_mask[:, :, None].repeat(1, 1, 16)
fg_scores_mask

tensor([[[1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]],

        [[1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [1., 1., 1., 1

In [169]:
target_scores = torch.where(fg_scores_mask > 0, target_scores, 0)
target_scores

tensor([[[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
         [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
         [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]],

        [[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
         [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
         [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
         [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
         [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]])

In [170]:
target_scores.size()

torch.Size([2, 8, 16])

In [171]:
# Normalize
align_metric *= mask_pos
align_metric

tensor([[[ 0.,  0.,  0.,  0.,  0.,  0.,  0., 10.],
         [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
         [ 7.,  8.,  0.,  0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  7.,  0.,  0.,  0.,  0.,  0.]],

        [[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
         [ 0.,  4.,  0.,  0.,  0.,  0.,  0.,  0.],
         [ 7.,  0.,  0.,  0.,  0.,  9.,  1.,  1.],
         [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]]])

In [172]:
pos_align_metrics = align_metric.amax(axis=-1, keepdim=True)
pos_align_metrics

tensor([[[10.],
         [ 0.],
         [ 8.],
         [ 7.]],

        [[ 0.],
         [ 4.],
         [ 9.],
         [ 0.]]])

In [173]:
overlaps * mask_pos

tensor([[[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.3373],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.9434, 0.7546, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.8041, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000]],

        [[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.8320, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.4278, 0.0000, 0.9587, 0.0000, 0.0000, 0.5515, 0.8688, 0.9818],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000]]])

In [174]:
pos_overlaps = (overlaps * mask_pos).amax(axis=-1, keepdim=True)
pos_overlaps

tensor([[[0.3373],
         [0.0000],
         [0.9434],
         [0.8041]],

        [[0.0000],
         [0.8320],
         [0.9818],
         [0.0000]]])

In [175]:
align_metric * pos_overlaps / (pos_align_metrics + 1e-9)

tensor([[[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.3373],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.8255, 0.9434, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.8041, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000]],

        [[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.8320, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.7636, 0.0000, 0.0000, 0.0000, 0.0000, 0.9818, 0.1091, 0.1091],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000]]])

In [176]:
norm_align_metric = (align_metric * pos_overlaps / (pos_align_metrics + 1e-9)).amax(-2).unsqueeze(-1)
norm_align_metric

tensor([[[0.8255],
         [0.9434],
         [0.8041],
         [0.0000],
         [0.0000],
         [0.0000],
         [0.0000],
         [0.3373]],

        [[0.7636],
         [0.8320],
         [0.0000],
         [0.0000],
         [0.0000],
         [0.9818],
         [0.1091],
         [0.1091]]])

In [177]:

target_scores = target_scores * norm_align_metric

In [178]:
target_scores[0]

tensor([[0.0000, 0.8255, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.9434, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.8041, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.00

In [179]:
target_bboxes.size(), target_scores.size(), fg_mask.size()

(torch.Size([2, 8, 4]), torch.Size([2, 8, 16]), torch.Size([2, 8]))

In [180]:

def autopad(k, p=None, d=1):  # kernel, padding, dilation
    """Pad to 'same' shape outputs."""
    if d > 1:
        k = d * (k - 1) + 1 if isinstance(k, int) else [d * (x - 1) + 1 for x in k]  # actual kernel-size
    if p is None:
        p = k // 2 if isinstance(k, int) else [x // 2 for x in k]  # auto-pad
    return p

class Conv(nn.Module):
    """Standard convolution with args(ch_in, ch_out, kernel, stride, padding, groups, dilation, activation)."""
    default_act = nn.SiLU()  # default activation

    def __init__(self, c1, c2, k=1, s=1, p=None, g=1, d=1, act=True):
        """Initialize Conv layer with given arguments including activation."""
        super().__init__()
        self.conv = nn.Conv2d(c1, c2, k, s, autopad(k, p, d), groups=g, dilation=d, bias=False)
        self.bn = nn.BatchNorm2d(c2)
        self.act = self.default_act if act is True else act if isinstance(act, nn.Module) else nn.Identity()

    def forward(self, x):
        """Apply convolution, batch normalization and activation to input tensor."""
        return self.act(self.bn(self.conv(x)))

    def forward_fuse(self, x):
        """Perform transposed convolution of 2D data."""
        return self.act(self.conv(x))

In [181]:
batch_size = 2
feats = [torch.rand((batch_size, 256, 80, 80)), 
         torch.rand((batch_size, 512, 40, 40)), 
         torch.rand((batch_size, 512, 20, 20))]

s = 640
strides = torch.tensor([s / x.shape[-2] for x in feats])
strides

tensor([ 8., 16., 32.])

In [182]:


def autopad(k, p=None, d=1):  # kernel, padding, dilation
    """Pad to 'same' shape outputs."""
    if d > 1:
        k = d * (k - 1) + 1 if isinstance(k, int) else [d * (x - 1) + 1 for x in k]  # actual kernel-size
    if p is None:
        p = k // 2 if isinstance(k, int) else [x // 2 for x in k]  # auto-pad
    return p

class Conv(nn.Module):
    """Standard convolution with args(ch_in, ch_out, kernel, stride, padding, groups, dilation, activation)."""
    default_act = nn.SiLU()  # default activation

    def __init__(self, c1, c2, k=1, s=1, p=None, g=1, d=1, act=True):
        """Initialize Conv layer with given arguments including activation."""
        super().__init__()
        self.conv = nn.Conv2d(c1, c2, k, s, autopad(k, p, d), groups=g, dilation=d, bias=False)
        self.bn = nn.BatchNorm2d(c2)
        self.act = self.default_act if act is True else act if isinstance(act, nn.Module) else nn.Identity()

    def forward(self, x):
        """Apply convolution, batch normalization and activation to input tensor."""
        return self.act(self.bn(self.conv(x)))

    def forward_fuse(self, x):
        """Perform transposed convolution of 2D data."""
        return self.act(self.conv(x))
    

ch = (256, 512, 512)

reg_max = 16 
nc = 16

c2, c3 = max((16, ch[0] // 4, reg_max * 4)), max(ch[0], min(nc, 100))

print(c2, c3)

cv2 = nn.ModuleList(
    nn.Sequential(
        Conv(x, c2, 3), 
        Conv(c2, c2, 3),
        nn.Conv2d(c2, 4 * reg_max, 1)) for x in ch)

cv3 = nn.ModuleList(
    nn.Sequential(
        Conv(x, c3, 3), 
        Conv(c3, c3, 3),
        nn.Conv2d(c3, nc, 1)) for x in ch)


for i in range(len(ch)):
    print(cv2[i](feats[i]).size(),cv3[i](feats[i]).size())
    print(torch.cat((cv2[i](feats[i]), cv3[i](feats[i])), 1).size())
    feats[i] = torch.cat((cv2[i](feats[i]), cv3[i](feats[i])), 1)



64 256
torch.Size([2, 64, 80, 80]) torch.Size([2, 16, 80, 80])
torch.Size([2, 80, 80, 80])
torch.Size([2, 64, 40, 40]) torch.Size([2, 16, 40, 40])
torch.Size([2, 80, 40, 40])
torch.Size([2, 64, 20, 20]) torch.Size([2, 16, 20, 20])
torch.Size([2, 80, 20, 20])


In [183]:

no = nc + reg_max * 4 
no

80

In [184]:
pred_distri, pred_scores = torch.cat([xi.view(feats[0].shape[0], no, -1) for xi in feats], 2).split((reg_max * 4, nc), 1)
pred_distri.size(), pred_scores.size()

(torch.Size([2, 64, 8400]), torch.Size([2, 16, 8400]))

In [185]:
pred_distri = pred_distri.permute(0, 2, 1).contiguous()
pred_scores = pred_scores.permute(0, 2, 1).contiguous()
pred_distri.size(), pred_scores.size()

(torch.Size([2, 8400, 64]), torch.Size([2, 8400, 16]))

In [186]:
pred_distri = torch.rand((2, 8, 16 * 4))
pred_scores = torch.rand((2, 8, 16))
pred_distri.size(), pred_scores.size()

(torch.Size([2, 8, 64]), torch.Size([2, 8, 16]))

In [187]:
bce = nn.BCEWithLogitsLoss(reduction='none')
print(bce(pred_scores, target_scores).sum())
print(-(target_scores * pred_scores.sigmoid().log() + (1 - target_scores) * (1 - pred_scores.sigmoid()).log()).sum())

tensor(245.5353)
tensor(245.5353)


In [188]:
def make_anchors(feats, strides, grid_cell_offset=0.5):
    """Generate anchors from features."""
    anchor_points, stride_tensor = [], []
    assert feats is not None
    dtype, device = feats[0].dtype, feats[0].device
    for i, stride in enumerate(strides):
        _, _, h, w = feats[i].shape
        sx = torch.arange(end=w, device=device, dtype=dtype) + grid_cell_offset  # shift x
        sy = torch.arange(end=h, device=device, dtype=dtype) + grid_cell_offset  # shift y
        sy, sx = torch.meshgrid(sy, sx, indexing='ij') 
        anchor_points.append(torch.stack((sx, sy), -1).view(-1, 2))
        stride_tensor.append(torch.full((h * w, 1), stride, dtype=dtype, device=device))
    return torch.cat(anchor_points), torch.cat(stride_tensor)

anchor_points, stride_tensor = make_anchors(feats, strides, 0.5)

print(anchor_points.size())
print(stride_tensor.size())

anchor_points = torch.rand((8, 2))
stride_tensor = torch.randint(8, 32, (8, 1)).float()

print(anchor_points.size())
print(stride_tensor.size())

def bbox2dist(anchor_points, bbox, reg_max):
    """Transform bbox(xyxy) to dist(ltrb)."""
    x1y1, x2y2 = bbox.chunk(2, -1)
    return torch.cat((anchor_points - x1y1, x2y2 - anchor_points), -1).clamp_(0, reg_max - 0.01)



torch.Size([8400, 2])
torch.Size([8400, 1])
torch.Size([8, 2])
torch.Size([8, 1])


In [189]:
target_bboxes[0, 0]

tensor([ 59,  10,  31, 115])

In [190]:
stride_tensor[0]

tensor([9.])

In [191]:
anchor_points[0]

tensor([0.9997, 0.5832])

In [192]:
target_bboxes = target_bboxes.float()
target_bboxes /= stride_tensor
target_bboxes[0, 0]

tensor([ 6.5556,  1.1111,  3.4444, 12.7778])

In [193]:
target_ltrb = bbox2dist(anchor_points, target_bboxes, 16 - 1)

target_ltrb[0, 0]

tensor([ 0.0000,  0.0000,  2.4448, 12.1946])

In [194]:
pred_distri.size(), target_ltrb.size()

(torch.Size([2, 8, 64]), torch.Size([2, 8, 4]))

In [195]:
pred_distri[fg_mask.bool()].size(), target_ltrb[fg_mask.bool()].size()

(torch.Size([10, 64]), torch.Size([10, 4]))

In [196]:
import torch.nn.functional as F

In [197]:
pred_distri[fg_mask.bool()].view(-1, 15 + 1).size()

torch.Size([40, 16])

In [198]:
target_ltrb[fg_mask.bool()].long()[0,:]

tensor([ 0,  0,  2, 12])

In [199]:
(target_ltrb[fg_mask.bool()].long() + 1)[0,:]

tensor([ 1,  1,  3, 13])

In [200]:
((target_ltrb[fg_mask.bool()].long() + 1) - target_ltrb[fg_mask.bool()])[0,:]

tensor([1.0000, 1.0000, 0.5552, 0.8054])

In [212]:
tl = target_ltrb[fg_mask.bool()].long()  # target left
tr = tl + 1  # target right
wl = tr - target_ltrb[fg_mask.bool()]  # weight left
wr = 1 - wl  # weight right

In [213]:
target_ltrb[fg_mask.bool()][0, :]

tensor([ 0.0000,  0.0000,  2.4448, 12.1946])

In [214]:
tl[0, :], wl[0, :]

(tensor([ 0,  0,  2, 12]), tensor([1.0000, 1.0000, 0.5552, 0.8054]))

In [217]:
tr[0, :], wr[0, :]

(tensor([ 1,  1,  3, 13]), tensor([0.0000, 0.0000, 0.4448, 0.1946]))

In [205]:
pred_distri[fg_mask.bool()].view(-1, 15 + 1).size()


torch.Size([40, 16])

In [224]:
(-(pred_distri[fg_mask.bool()].view(-1, 15 + 1).softmax(-1).log() * F.one_hot(tl.view(-1), num_classes=16)).sum(-1).view(tl.shape) * wl + \
(-(pred_distri[fg_mask.bool()].view(-1, 15 + 1).softmax(-1).log() * F.one_hot(tr.view(-1), num_classes=16)).sum(-1)).view(tl.shape) * wr).mean(-1, keepdim=True)

tensor([[2.7523],
        [2.6878],
        [2.9389],
        [2.8822],
        [2.8924],
        [2.8146],
        [2.8061],
        [2.7107],
        [2.8432],
        [2.6574]])

In [225]:
(F.cross_entropy(pred_distri[fg_mask.bool()].view(-1, 15 + 1), tl.view(-1), reduction='none').view(tl.shape) * wl +
F.cross_entropy(pred_distri[fg_mask.bool()].view(-1, 15 + 1), tr.view(-1), reduction='none').view(tl.shape) * wr).mean(-1, keepdim=True)

tensor([[2.7523],
        [2.6878],
        [2.9389],
        [2.8822],
        [2.8924],
        [2.8146],
        [2.8061],
        [2.7107],
        [2.8432],
        [2.6574]])

In [ ]:
def dist2bbox(distance, anchor_points, xywh=True, dim=-1):
    """Transform distance(ltrb) to box(xywh or xyxy)."""
    lt, rb = distance.chunk(2, dim)
    x1y1 = anchor_points - lt
    x2y2 = anchor_points + rb
    if xywh:
        c_xy = (x1y1 + x2y2) / 2
        wh = x2y2 - x1y1
        return torch.cat((c_xy, wh), dim)  # xywh bbox
    return torch.cat((x1y1, x2y2), dim)  # xyxy bbox

def bbox_decode(anchor_points, pred_dist):
    """Decode predicted object bounding box coordinates from anchor points and distribution."""

    b, a, c = pred_dist.shape  # batch, anchors, channels
    pred_dist = pred_dist.view(b, a, 4, c // 4).softmax(3).matmul()
  
    return dist2bbox(pred_dist, anchor_points, xywh=False)




In [ ]:

def bbox_iou(box1, box2, xywh=True, GIoU=False, DIoU=False, CIoU=False, eps=1e-7):

    # Get the coordinates of bounding boxes
    if xywh:  # transform from xywh to xyxy
        (x1, y1, w1, h1), (x2, y2, w2, h2) = box1.chunk(4, -1), box2.chunk(4, -1)
        w1_, h1_, w2_, h2_ = w1 / 2, h1 / 2, w2 / 2, h2 / 2
        b1_x1, b1_x2, b1_y1, b1_y2 = x1 - w1_, x1 + w1_, y1 - h1_, y1 + h1_
        b2_x1, b2_x2, b2_y1, b2_y2 = x2 - w2_, x2 + w2_, y2 - h2_, y2 + h2_
    else:  # x1, y1, x2, y2 = box1
        b1_x1, b1_y1, b1_x2, b1_y2 = box1.chunk(4, -1)
        b2_x1, b2_y1, b2_x2, b2_y2 = box2.chunk(4, -1)
        w1, h1 = b1_x2 - b1_x1, b1_y2 - b1_y1 + eps
        w2, h2 = b2_x2 - b2_x1, b2_y2 - b2_y1 + eps
    # Intersection area
    inter = (b1_x2.minimum(b2_x2) - b1_x1.maximum(b2_x1)).clamp_(0) * \
            (b1_y2.minimum(b2_y2) - b1_y1.maximum(b2_y1)).clamp_(0)

    # Union Area
    union = w1 * h1 + w2 * h2 - inter + eps

    # IoU
    iou = inter / union
    if CIoU or DIoU or GIoU:
        cw = b1_x2.maximum(b2_x2) - b1_x1.minimum(b2_x1)  # convex (smallest enclosing box) width
        ch = b1_y2.maximum(b2_y2) - b1_y1.minimum(b2_y1)  # convex height
        if CIoU or DIoU:  # Distance or Complete IoU 
            c2 = cw ** 2 + ch ** 2 + eps  # convex diagonal squared
            rho2 = ((b2_x1 + b2_x2 - b1_x1 - b1_x2) ** 2 + (b2_y1 + b2_y2 - b1_y1 - b1_y2) ** 2) / 4  # center dist ** 2
            if CIoU: 
                v = (4 / math.pi ** 2) * (torch.atan(w2 / h2) - torch.atan(w1 / h1)).pow(2)
                with torch.no_grad():
                    alpha = v / (v - iou + (1 + eps))
                return iou - (rho2 / c2 + v * alpha)  # CIoU
            return iou - rho2 / c2  # DIoU
        c_area = cw * ch + eps  # convex area
        return iou - (c_area - union) / c_area 
    return iou  # IoU

pd_bboxes = torch.rand((2, 8, 4))

target_scores_sum = max(target_scores.sum(), 1)

weight = target_scores.sum(-1)[fg_mask.bool()].unsqueeze(-1)
iou = bbox_iou(pd_bboxes[fg_mask.bool()], target_bboxes[fg_mask.bool()], xywh=False, CIoU=True)
loss_iou = ((1.0 - iou) * weight).sum() / target_scores_sum
loss_iou

In [ ]:
from ultralytics import YOLO

In [ ]:
model = YOLO('yolov8n.yaml')
model.train()

In [ ]:
import math
import cv2 as cv
import numpy as np

import torch
import torch.nn as nn

In [ ]:
def autopad(k, p=None, d=1):  # kernel, padding, dilation
    """Pad to 'same' shape outputs."""
    if d > 1:
        k = d * (k - 1) + 1 if isinstance(k, int) else [d * (x - 1) + 1 for x in k]  # actual kernel-size
    if p is None:
        p = k // 2 if isinstance(k, int) else [x // 2 for x in k]  # auto-pad
    return p

class Conv(nn.Module):
    """Standard convolution with args(ch_in, ch_out, kernel, stride, padding, groups, dilation, activation)."""
    default_act = nn.SiLU()  # default activation

    def __init__(self, c1, c2, k=1, s=1, p=None, g=1, d=1, act=True):
        """Initialize Conv layer with given arguments including activation."""
        super().__init__()
        self.conv = nn.Conv2d(c1, c2, k, s, autopad(k, p, d), groups=g, dilation=d, bias=False)
        self.bn = nn.BatchNorm2d(c2)
        self.act = self.default_act if act is True else act if isinstance(act, nn.Module) else nn.Identity()

    def forward(self, x):
        """Apply convolution, batch normalization and activation to input tensor."""
        return self.act(self.bn(self.conv(x)))

    def forward_fuse(self, x):
        """Perform transposed convolution of 2D data."""
        return self.act(self.conv(x))
    

    

In [ ]:


class ShuffleChannel(nn.Module):

    def __init__(self, g) -> None:
        super().__init__()

        self.g = g

    def forward(self, x):
        b, c, h, w = x.size()

        x = x.view()
        x = torch.transpose()
        x = x.view()

        return x

class ShuffleConv(nn.Module):


    def __init__(self, c1, c2, k=1, s=1, p=None, g=1, d=1) -> None:
        super().__init__()

        self.GConv1 = nn.Sequential(
            nn.Conv2d(),
            nn.BatchNorm2d(),
            nn.SiLU(),
        )

        self.DWConv1 = nn.Sequential(
            nn.Conv2d(),
            nn.BatchNorm2d(),
        )

        self.GConv2 = nn.Sequential(
            nn.Conv2d(),
            nn.BatchNorm2d(),
        )

        self.shorcut = nn.Sequential(
            nn.AvgPool2d()
        )

        self.shuffleChannel = ShuffleChannel(g)

    def forward(self, x):
        y = self.GConv1(x)
        y = self.shuffleChannel(y)
        y = self.DWConv1(y)
        y = self.GConv2(y)
        
        short = self.shorcut(x)
        torch.cat([y, short], dim=1)

        return y
        


In [ ]:
class DFL(nn.Module):

    def __init__(self, c1=16):

        super().__init__()
        self.conv = nn.Conv2d(c1, 1, 1, bias=False).requires_grad_(False)
        x = torch.arange(c1, dtype=torch.float)
        self.conv.weight.data[:] = nn.Parameter(x.view(1, c1, 1, 1))
        self.c1 = c1

    def forward(self, x):
        b, c, a = x.shape
        return self.conv(x.view(b, 4, self.c1, a).transpose(2, 1).softmax(1)).view(b, 4, a)
    
class Detect(nn.Module):
    """YOLOv8 Detect head for detection models."""
    dynamic = False  # force grid reconstruction
    export = False  # export mode
    shape = None
    anchors = torch.empty(0)  # init
    strides = torch.empty(0)  # init

    def __init__(self, nc=80, ch=()):  # detection layer
        super().__init__()
        self.nc = nc  # number of classes
        self.nl = len(ch)  # number of detection layers
        self.reg_max = 16  # DFL channels (ch[0] // 16 to scale 4/8/12/16/20 for n/s/m/l/x)
        self.no = nc + self.reg_max * 4  # number of outputs per anchor
        self.stride = torch.zeros(self.nl)  # strides computed during build
        c2, c3 = max((16, ch[0] // 4, self.reg_max * 4)), max(ch[0], self.nc)  # channels
        self.cv2 = nn.ModuleList(
            nn.Sequential(Conv(x, c2, 3), Conv(c2, c2, 3), nn.Conv2d(c2, 4 * self.reg_max, 1)) for x in ch)
        self.cv3 = nn.ModuleList(nn.Sequential(Conv(x, c3, 3), Conv(c3, c3, 3), nn.Conv2d(c3, self.nc, 1)) for x in ch)
        self.dfl = DFL(self.reg_max) if self.reg_max > 1 else nn.Identity()

    def forward(self, x):
        """Concatenates and returns predicted bounding boxes and class probabilities."""
        shape = x[0].shape  # BCHW
        for i in range(self.nl):
            x[i] = torch.cat((self.cv2[i](x[i]), self.cv3[i](x[i])), 1)
        if self.training:
            return x
        elif self.dynamic or self.shape != shape:
            self.anchors, self.strides = (x.transpose(0, 1) for x in make_anchors(x, self.stride, 0.5))
            self.shape = shape

        x_cat = torch.cat([xi.view(shape[0], self.no, -1) for xi in x], 2)
        if self.export and self.format in ('saved_model', 'pb', 'tflite', 'edgetpu', 'tfjs'):  # avoid TF FlexSplitV ops
            box = x_cat[:, :self.reg_max * 4]
            cls = x_cat[:, self.reg_max * 4:]
        else:
            box, cls = x_cat.split((self.reg_max * 4, self.nc), 1)
        dbox = dist2bbox(self.dfl(box), self.anchors.unsqueeze(0), xywh=True, dim=1) * self.strides
        y = torch.cat((dbox, cls.sigmoid()), 1)
        return y if self.export else (y, x)

    def bias_init(self):
        """Initialize Detect() biases, WARNING: requires stride availability."""
        m = self  # self.model[-1]  # Detect() module
        # cf = torch.bincount(torch.tensor(np.concatenate(dataset.labels, 0)[:, 0]).long(), minlength=nc) + 1
        # ncf = math.log(0.6 / (m.nc - 0.999999)) if cf is None else torch.log(cf / cf.sum())  # nominal class frequency
        for a, b, s in zip(m.cv2, m.cv3, m.stride):  # from
            a[-1].bias.data[:] = 1.0  # box
            b[-1].bias.data[:m.nc] = math.log(5 / m.nc / (640 / s) ** 2)  # cls (.01 objects, 80 classes, 640 img)

In [ ]:
class SPPF(nn.Module):
    """Spatial Pyramid Pooling - Fast (SPPF) layer for YOLOv5 by Glenn Jocher."""

    def __init__(self, c1, c2, k=5):  # equivalent to SPP(k=(5, 9, 13))
        super().__init__()
        c_ = c1 // 2  # hidden channels
        self.cv1 = Conv(c1, c_, 1, 1)
        self.cv2 = Conv(c_ * 4, c2, 1, 1)
        self.m = nn.MaxPool2d(kernel_size=k, stride=1, padding=k // 2)

    def forward(self, x):
        """Forward pass through Ghost Convolution block."""
        x = self.cv1(x)
        y1 = self.m(x)
        y2 = self.m(y1)
        return self.cv2(torch.cat((x, y1, y2, self.m(y2)), 1))

In [ ]:
class Bottleneck(nn.Module):
    """Standard bottleneck."""

    def __init__(self, c1, c2, shortcut=True, g=1, k=(3, 3), e=0.5):  # ch_in, ch_out, shortcut, groups, kernels, expand
        super().__init__()
        c_ = int(c2 * e)  # hidden channels
        self.cv1 = Conv(c1, c_, k[0], 1)
        self.cv2 = Conv(c_, c2, k[1], 1, g=g)
        self.add = shortcut and c1 == c2

    def forward(self, x):
        """'forward()' applies the YOLOv5 FPN to input data."""
        return x + self.cv2(self.cv1(x)) if self.add else self.cv2(self.cv1(x))

class C2f(nn.Module):
    """CSP Bottleneck with 2 convolutions."""

    def __init__(self, c1, c2, n=1, shortcut=False, g=1, e=0.5):  # ch_in, ch_out, number, shortcut, groups, expansion
        super().__init__()
        self.c = int(c2 * e)  # hidden channels
        self.cv1 = Conv(c1, 2 * self.c, 1, 1)
        self.cv2 = Conv((2 + n) * self.c, c2, 1)  # optional act=FReLU(c2)
        self.m = nn.ModuleList(Bottleneck(self.c, self.c, shortcut, g, k=((3, 3), (3, 3)), e=1.0) for _ in range(n))

    def forward(self, x):
        """Forward pass through C2f layer."""
        y = list(self.cv1(x).chunk(2, 1))
        y.extend(m(y[-1]) for m in self.m)
        return self.cv2(torch.cat(y, 1))

    def forward_split(self, x):
        """Forward pass using split() instead of chunk()."""
        y = list(self.cv1(x).split((self.c, self.c), 1))
        y.extend(m(y[-1]) for m in self.m)
        return self.cv2(torch.cat(y, 1))

In [ ]:



def preprocess(self, targets, batch_size, scale_tensor):
    """Preprocesses the target counts and matches with the input batch size to output a tensor."""
    if targets.shape[0] == 0:
        out = torch.zeros(batch_size, 0, 5, device=self.device)
    else:
        i = targets[:, 0]  # image index
        _, counts = i.unique(return_counts=True)
        counts = counts.to(dtype=torch.int32)
        out = torch.zeros(batch_size, counts.max(), 5, device=self.device)
        for j in range(batch_size):
            matches = i == j
            n = matches.sum()
            if n:
                out[j, :n] = targets[matches, 1:]
        out[..., 1:5] = xywh2xyxy(out[..., 1:5].mul_(scale_tensor))
    return out

def bbox_decode(self, anchor_points, pred_dist):
    """Decode predicted object bounding box coordinates from anchor points and distribution."""
    if self.use_dfl:
        b, a, c = pred_dist.shape  # batch, anchors, channels
        pred_dist = pred_dist.view(b, a, 4, c // 4).softmax(3).matmul(self.proj.type(pred_dist.dtype))
        # pred_dist = pred_dist.view(b, a, c // 4, 4).transpose(2,3).softmax(3).matmul(self.proj.type(pred_dist.dtype))
        # pred_dist = (pred_dist.view(b, a, c // 4, 4).softmax(2) * self.proj.type(pred_dist.dtype).view(1, 1, -1, 1)).sum(2)
    return dist2bbox(pred_dist, anchor_points, xywh=False)




In [ ]:

def compute_ap(recall, precision):
    """
    Compute the average precision (AP) given the recall and precision curves.

    Arguments:
        recall (list): The recall curve.
        precision (list): The precision curve.

    Returns:
        (float): Average precision.
        (np.ndarray): Precision envelope curve.
        (np.ndarray): Modified recall curve with sentinel values added at the beginning and end.
    """

    # Append sentinel values to beginning and end
    mrec = np.concatenate(([0.0], recall, [1.0]))
    mpre = np.concatenate(([1.0], precision, [0.0]))

    # Compute the precision envelope
    mpre = np.flip(np.maximum.accumulate(np.flip(mpre)))

    # Integrate area under curve
    method = 'interp'  # methods: 'continuous', 'interp'
    if method == 'interp':
        x = np.linspace(0, 1, 101)  # 101-point interp (COCO)
        ap = np.trapz(np.interp(x, mrec, mpre), x)  # integrate
    else:  # 'continuous'
        i = np.where(mrec[1:] != mrec[:-1])[0]  # points where x-axis (recall) changes
        ap = np.sum((mrec[i + 1] - mrec[i]) * mpre[i + 1])  # area under curve

    return ap, mpre, mrec


def ap_per_class(tp,
                 conf,
                 pred_cls,
                 target_cls,
                 plot=False,
                 on_plot=None,
                 save_dir=Path(),
                 names=(),
                 eps=1e-16,
                 prefix=''):
    """
    Computes the average precision per class for object detection evaluation.

    Args:
        tp (np.ndarray): Binary array indicating whether the detection is correct (True) or not (False).
        conf (np.ndarray): Array of confidence scores of the detections.
        pred_cls (np.ndarray): Array of predicted classes of the detections.
        target_cls (np.ndarray): Array of true classes of the detections.
        plot (bool, optional): Whether to plot PR curves or not. Defaults to False.
        on_plot (func, optional): A callback to pass plots path and data when they are rendered. Defaults to None.
        save_dir (Path, optional): Directory to save the PR curves. Defaults to an empty path.
        names (tuple, optional): Tuple of class names to plot PR curves. Defaults to an empty tuple.
        eps (float, optional): A small value to avoid division by zero. Defaults to 1e-16.
        prefix (str, optional): A prefix string for saving the plot files. Defaults to an empty string.

    Returns:
        (tuple): A tuple of six arrays and one array of unique classes, where:
            tp (np.ndarray): True positive counts for each class.
            fp (np.ndarray): False positive counts for each class.
            p (np.ndarray): Precision values at each confidence threshold.
            r (np.ndarray): Recall values at each confidence threshold.
            f1 (np.ndarray): F1-score values at each confidence threshold.
            ap (np.ndarray): Average precision for each class at different IoU thresholds.
            unique_classes (np.ndarray): An array of unique classes that have data.

    """

    # Sort by objectness
    i = np.argsort(-conf)
    tp, conf, pred_cls = tp[i], conf[i], pred_cls[i]

    # Find unique classes
    unique_classes, nt = np.unique(target_cls, return_counts=True)
    nc = unique_classes.shape[0]  # number of classes, number of detections

    # Create Precision-Recall curve and compute AP for each class
    px, py = np.linspace(0, 1, 1000), []  # for plotting
    ap, p, r = np.zeros((nc, tp.shape[1])), np.zeros((nc, 1000)), np.zeros((nc, 1000))
    for ci, c in enumerate(unique_classes):
        i = pred_cls == c
        n_l = nt[ci]  # number of labels
        n_p = i.sum()  # number of predictions
        if n_p == 0 or n_l == 0:
            continue

        # Accumulate FPs and TPs
        fpc = (1 - tp[i]).cumsum(0)
        tpc = tp[i].cumsum(0)

        # Recall
        recall = tpc / (n_l + eps)  # recall curve
        r[ci] = np.interp(-px, -conf[i], recall[:, 0], left=0)  # negative x, xp because xp decreases

        # Precision
        precision = tpc / (tpc + fpc)  # precision curve
        p[ci] = np.interp(-px, -conf[i], precision[:, 0], left=1)  # p at pr_score

        # AP from recall-precision curve
        for j in range(tp.shape[1]):
            ap[ci, j], mpre, mrec = compute_ap(recall[:, j], precision[:, j])
            if plot and j == 0:
                py.append(np.interp(px, mrec, mpre))  # precision at mAP@0.5

    # Compute F1 (harmonic mean of precision and recall)
    f1 = 2 * p * r / (p + r + eps)
    names = [v for k, v in names.items() if k in unique_classes]  # list: only classes that have data
    names = dict(enumerate(names))  # to dict
    if plot:
        plot_pr_curve(px, py, ap, save_dir / f'{prefix}PR_curve.png', names, on_plot=on_plot)
        plot_mc_curve(px, f1, save_dir / f'{prefix}F1_curve.png', names, ylabel='F1', on_plot=on_plot)
        plot_mc_curve(px, p, save_dir / f'{prefix}P_curve.png', names, ylabel='Precision', on_plot=on_plot)
        plot_mc_curve(px, r, save_dir / f'{prefix}R_curve.png', names, ylabel='Recall', on_plot=on_plot)

    i = smooth(f1.mean(0), 0.1).argmax()  # max F1 index
    p, r, f1 = p[:, i], r[:, i], f1[:, i]
    tp = (r * nt).round()  # true positives
    fp = (tp / (p + eps) - tp).round()  # false positives
    return tp, fp, p, r, f1, ap, unique_classes.astype(int)


class Metric(SimpleClass):
    """
        Class for computing evaluation metrics for YOLOv8 model.

        Attributes:
            p (list): Precision for each class. Shape: (nc,).
            r (list): Recall for each class. Shape: (nc,).
            f1 (list): F1 score for each class. Shape: (nc,).
            all_ap (list): AP scores for all classes and all IoU thresholds. Shape: (nc, 10).
            ap_class_index (list): Index of class for each AP score. Shape: (nc,).
            nc (int): Number of classes.

        Methods:
            ap50(): AP at IoU threshold of 0.5 for all classes. Returns: List of AP scores. Shape: (nc,) or [].
            ap(): AP at IoU thresholds from 0.5 to 0.95 for all classes. Returns: List of AP scores. Shape: (nc,) or [].
            mp(): Mean precision of all classes. Returns: Float.
            mr(): Mean recall of all classes. Returns: Float.
            map50(): Mean AP at IoU threshold of 0.5 for all classes. Returns: Float.
            map75(): Mean AP at IoU threshold of 0.75 for all classes. Returns: Float.
            map(): Mean AP at IoU thresholds from 0.5 to 0.95 for all classes. Returns: Float.
            mean_results(): Mean of results, returns mp, mr, map50, map.
            class_result(i): Class-aware result, returns p[i], r[i], ap50[i], ap[i].
            maps(): mAP of each class. Returns: Array of mAP scores, shape: (nc,).
            fitness(): Model fitness as a weighted combination of metrics. Returns: Float.
            update(results): Update metric attributes with new evaluation results.

        """

    def __init__(self) -> None:
        self.p = []  # (nc, )
        self.r = []  # (nc, )
        self.f1 = []  # (nc, )
        self.all_ap = []  # (nc, 10)
        self.ap_class_index = []  # (nc, )
        self.nc = 0

    @property
    def ap50(self):
        """
        Returns the Average Precision (AP) at an IoU threshold of 0.5 for all classes.

        Returns:
            (np.ndarray, list): Array of shape (nc,) with AP50 values per class, or an empty list if not available.
        """
        return self.all_ap[:, 0] if len(self.all_ap) else []

    @property
    def ap(self):
        """
        Returns the Average Precision (AP) at an IoU threshold of 0.5-0.95 for all classes.

        Returns:
            (np.ndarray, list): Array of shape (nc,) with AP50-95 values per class, or an empty list if not available.
        """
        return self.all_ap.mean(1) if len(self.all_ap) else []

    @property
    def mp(self):
        """
        Returns the Mean Precision of all classes.

        Returns:
            (float): The mean precision of all classes.
        """
        return self.p.mean() if len(self.p) else 0.0

    @property
    def mr(self):
        """
        Returns the Mean Recall of all classes.

        Returns:
            (float): The mean recall of all classes.
        """
        return self.r.mean() if len(self.r) else 0.0

    @property
    def map50(self):
        """
        Returns the mean Average Precision (mAP) at an IoU threshold of 0.5.

        Returns:
            (float): The mAP50 at an IoU threshold of 0.5.
        """
        return self.all_ap[:, 0].mean() if len(self.all_ap) else 0.0

    @property
    def map75(self):
        """
        Returns the mean Average Precision (mAP) at an IoU threshold of 0.75.

        Returns:
            (float): The mAP50 at an IoU threshold of 0.75.
        """
        return self.all_ap[:, 5].mean() if len(self.all_ap) else 0.0

    @property
    def map(self):
        """
        Returns the mean Average Precision (mAP) over IoU thresholds of 0.5 - 0.95 in steps of 0.05.

        Returns:
            (float): The mAP over IoU thresholds of 0.5 - 0.95 in steps of 0.05.
        """
        return self.all_ap.mean() if len(self.all_ap) else 0.0

    def mean_results(self):
        """Mean of results, return mp, mr, map50, map."""
        return [self.mp, self.mr, self.map50, self.map]

    def class_result(self, i):
        """class-aware result, return p[i], r[i], ap50[i], ap[i]."""
        return self.p[i], self.r[i], self.ap50[i], self.ap[i]

    @property
    def maps(self):
        """mAP of each class."""
        maps = np.zeros(self.nc) + self.map
        for i, c in enumerate(self.ap_class_index):
            maps[c] = self.ap[i]
        return maps

    def fitness(self):
        """Model fitness as a weighted combination of metrics."""
        w = [0.0, 0.0, 0.1, 0.9]  # weights for [P, R, mAP@0.5, mAP@0.5:0.95]
        return (np.array(self.mean_results()) * w).sum()

    def update(self, results):
        """
        Args:
            results (tuple): A tuple of (p, r, ap, f1, ap_class)
        """
        self.p, self.r, self.f1, self.all_ap, self.ap_class_index = results


class DetMetrics(SimpleClass):
    """
    This class is a utility class for computing detection metrics such as precision, recall, and mean average precision
    (mAP) of an object detection model.

    Args:
        save_dir (Path): A path to the directory where the output plots will be saved. Defaults to current directory.
        plot (bool): A flag that indicates whether to plot precision-recall curves for each class. Defaults to False.
        on_plot (func): An optional callback to pass plots path and data when they are rendered. Defaults to None.
        names (tuple of str): A tuple of strings that represents the names of the classes. Defaults to an empty tuple.

    Attributes:
        save_dir (Path): A path to the directory where the output plots will be saved.
        plot (bool): A flag that indicates whether to plot the precision-recall curves for each class.
        on_plot (func): An optional callback to pass plots path and data when they are rendered.
        names (tuple of str): A tuple of strings that represents the names of the classes.
        box (Metric): An instance of the Metric class for storing the results of the detection metrics.
        speed (dict): A dictionary for storing the execution time of different parts of the detection process.

    Methods:
        process(tp, conf, pred_cls, target_cls): Updates the metric results with the latest batch of predictions.
        keys: Returns a list of keys for accessing the computed detection metrics.
        mean_results: Returns a list of mean values for the computed detection metrics.
        class_result(i): Returns a list of values for the computed detection metrics for a specific class.
        maps: Returns a dictionary of mean average precision (mAP) values for different IoU thresholds.
        fitness: Computes the fitness score based on the computed detection metrics.
        ap_class_index: Returns a list of class indices sorted by their average precision (AP) values.
        results_dict: Returns a dictionary that maps detection metric keys to their computed values.
    """

    def __init__(self, save_dir=Path('.'), plot=False, on_plot=None, names=()) -> None:
        self.save_dir = save_dir
        self.plot = plot
        self.on_plot = on_plot
        self.names = names
        self.box = Metric()
        self.speed = {'preprocess': 0.0, 'inference': 0.0, 'loss': 0.0, 'postprocess': 0.0}

    def process(self, tp, conf, pred_cls, target_cls):
        """Process predicted results for object detection and update metrics."""
        results = ap_per_class(tp,
                               conf,
                               pred_cls,
                               target_cls,
                               plot=self.plot,
                               save_dir=self.save_dir,
                               names=self.names,
                               on_plot=self.on_plot)[2:]
        self.box.nc = len(self.names)
        self.box.update(results)

    @property
    def keys(self):
        """Returns a list of keys for accessing specific metrics."""
        return ['metrics/precision(B)', 'metrics/recall(B)', 'metrics/mAP50(B)', 'metrics/mAP50-95(B)']

    def mean_results(self):
        """Calculate mean of detected objects & return precision, recall, mAP50, and mAP50-95."""
        return self.box.mean_results()

    def class_result(self, i):
        """Return the result of evaluating the performance of an object detection model on a specific class."""
        return self.box.class_result(i)

    @property
    def maps(self):
        """Returns mean Average Precision (mAP) scores per class."""
        return self.box.maps

    @property
    def fitness(self):
        """Returns the fitness of box object."""
        return self.box.fitness()

    @property
    def ap_class_index(self):
        """Returns the average precision index per class."""
        return self.box.ap_class_index

    @property
    def results_dict(self):
        """Returns dictionary of computed performance metrics and statistics."""
        return dict(zip(self.keys + ['fitness'], self.mean_results() + [self.fitness]))


In [ ]:
def non_max_suppression(
        prediction,
        conf_thres=0.25,
        iou_thres=0.45,
        classes=None,
        agnostic=False,
        multi_label=False,
        labels=(),
        max_det=300,
        nc=0,  # number of classes (optional)
        max_time_img=0.05,
        max_nms=30000,
        max_wh=7680,
):
    """
    Perform non-maximum suppression (NMS) on a set of boxes, with support for masks and multiple labels per box.

    Arguments:
        prediction (torch.Tensor): A tensor of shape (batch_size, num_classes + 4 + num_masks, num_boxes)
            containing the predicted boxes, classes, and masks. The tensor should be in the format
            output by a model, such as YOLO.
        conf_thres (float): The confidence threshold below which boxes will be filtered out.
            Valid values are between 0.0 and 1.0.
        iou_thres (float): The IoU threshold below which boxes will be filtered out during NMS.
            Valid values are between 0.0 and 1.0.
        classes (List[int]): A list of class indices to consider. If None, all classes will be considered.
        agnostic (bool): If True, the model is agnostic to the number of classes, and all
            classes will be considered as one.
        multi_label (bool): If True, each box may have multiple labels.
        labels (List[List[Union[int, float, torch.Tensor]]]): A list of lists, where each inner
            list contains the apriori labels for a given image. The list should be in the format
            output by a dataloader, with each label being a tuple of (class_index, x1, y1, x2, y2).
        max_det (int): The maximum number of boxes to keep after NMS.
        nc (int): (optional) The number of classes output by the model. Any indices after this will be considered masks.
        max_time_img (float): The maximum time (seconds) for processing one image.
        max_nms (int): The maximum number of boxes into torchvision.ops.nms().
        max_wh (int): The maximum box width and height in pixels

    Returns:
        (List[torch.Tensor]): A list of length batch_size, where each element is a tensor of
            shape (num_boxes, 6 + num_masks) containing the kept boxes, with columns
            (x1, y1, x2, y2, confidence, class, mask1, mask2, ...).
    """

    # Checks
    assert 0 <= conf_thres <= 1, f'Invalid Confidence threshold {conf_thres}, valid values are between 0.0 and 1.0'
    assert 0 <= iou_thres <= 1, f'Invalid IoU {iou_thres}, valid values are between 0.0 and 1.0'
    if isinstance(prediction, (list, tuple)):  # YOLOv8 model in validation model, output = (inference_out, loss_out)
        prediction = prediction[0]  # select only inference output

    device = prediction.device
    mps = 'mps' in device.type  # Apple MPS
    if mps:  # MPS not fully supported yet, convert tensors to CPU before NMS
        prediction = prediction.cpu()
    bs = prediction.shape[0]  # batch size
    nc = nc or (prediction.shape[1] - 4)  # number of classes
    nm = prediction.shape[1] - nc - 4
    mi = 4 + nc  # mask start index
    xc = prediction[:, 4:mi].amax(1) > conf_thres  # candidates

    # Settings
    # min_wh = 2  # (pixels) minimum box width and height
    time_limit = 0.5 + max_time_img * bs  # seconds to quit after
    redundant = True  # require redundant detections
    multi_label &= nc > 1  # multiple labels per box (adds 0.5ms/img)
    merge = False  # use merge-NMS

    t = time.time()
    output = [torch.zeros((0, 6 + nm), device=prediction.device)] * bs
    for xi, x in enumerate(prediction):  # image index, image inference
        # Apply constraints
        # x[((x[:, 2:4] < min_wh) | (x[:, 2:4] > max_wh)).any(1), 4] = 0  # width-height
        x = x.transpose(0, -1)[xc[xi]]  # confidence

        # Cat apriori labels if autolabelling
        if labels and len(labels[xi]):
            lb = labels[xi]
            v = torch.zeros((len(lb), nc + nm + 5), device=x.device)
            v[:, :4] = lb[:, 1:5]  # box
            v[range(len(lb)), lb[:, 0].long() + 4] = 1.0  # cls
            x = torch.cat((x, v), 0)

        # If none remain process next image
        if not x.shape[0]:
            continue

        # Detections matrix nx6 (xyxy, conf, cls)
        box, cls, mask = x.split((4, nc, nm), 1)
        box = xywh2xyxy(box)  # center_x, center_y, width, height) to (x1, y1, x2, y2)
        if multi_label:
            i, j = (cls > conf_thres).nonzero(as_tuple=False).T
            x = torch.cat((box[i], x[i, 4 + j, None], j[:, None].float(), mask[i]), 1)
        else:  # best class only
            conf, j = cls.max(1, keepdim=True)
            x = torch.cat((box, conf, j.float(), mask), 1)[conf.view(-1) > conf_thres]

        # Filter by class
        if classes is not None:
            x = x[(x[:, 5:6] == torch.tensor(classes, device=x.device)).any(1)]

        # Apply finite constraint
        # if not torch.isfinite(x).all():
        #     x = x[torch.isfinite(x).all(1)]

        # Check shape
        n = x.shape[0]  # number of boxes
        if not n:  # no boxes
            continue
        x = x[x[:, 4].argsort(descending=True)[:max_nms]]  # sort by confidence and remove excess boxes

        # Batched NMS
        c = x[:, 5:6] * (0 if agnostic else max_wh)  # classes
        boxes, scores = x[:, :4] + c, x[:, 4]  # boxes (offset by class), scores
        i = torchvision.ops.nms(boxes, scores, iou_thres)  # NMS
        i = i[:max_det]  # limit detections
        if merge and (1 < n < 3E3):  # Merge NMS (boxes merged using weighted mean)
            # Update boxes as boxes(i,4) = weights(i,n) * boxes(n,4)
            iou = box_iou(boxes[i], boxes) > iou_thres  # iou matrix
            weights = iou * scores[None]  # box weights
            x[i, :4] = torch.mm(weights, x[:, :4]).float() / weights.sum(1, keepdim=True)  # merged boxes
            if redundant:
                i = i[iou.sum(1) > 1]  # require redundancy

        output[xi] = x[i]
        if mps:
            output[xi] = output[xi].to(device)
        if (time.time() - t) > time_limit:
            LOGGER.warning(f'WARNING ⚠️ NMS time limit {time_limit:.3f}s exceeded')
            break  # time limit exceeded

    return output


In [ ]:
def _process_batch(self, detections, labels):
    """
    Return correct prediction matrix
    Arguments:
        detections (array[N, 6]), x1, y1, x2, y2, conf, class
        labels (array[M, 5]), class, x1, y1, x2, y2
    Returns:
        correct (array[N, 10]), for 10 IoU levels
    """
    iou = box_iou(labels[:, 1:], detections[:, :4])
    correct = np.zeros((detections.shape[0], self.iouv.shape[0])).astype(bool)
    correct_class = labels[:, 0:1] == detections[:, 5]
    for i in range(len(self.iouv)):
        x = torch.where((iou >= self.iouv[i]) & correct_class)  # IoU > threshold and classes match
        if x[0].shape[0]:
            matches = torch.cat((torch.stack(x, 1), iou[x[0], x[1]][:, None]), 1).cpu().numpy()  # [label, detect, iou]
            if x[0].shape[0] > 1:
                matches = matches[matches[:, 2].argsort()[::-1]]
                matches = matches[np.unique(matches[:, 1], return_index=True)[1]]
                # matches = matches[matches[:, 2].argsort()[::-1]]
                matches = matches[np.unique(matches[:, 0], return_index=True)[1]]
            correct[matches[:, 1].astype(int), i] = True
    return torch.tensor(correct, dtype=torch.bool, device=detections.device)

In [ ]:


model = dict(type='ImageClassifier',
            data_preprocessor=dict(type='DataPreprocessor'),
            backbone=dict(type='ResNet', in_channels=3, out_channels = 3),
            neck=dict(type='GlobalAveragePooling'),
            head=dict(type='LinearClsHead',
                      loss=dict(type='CrossEntropyLoss'), 
                      num_classes=100, 
                      in_channels = 3),)

train_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(type='RandomResizedCrop', scale = 224),
    dict(type='PackInputs'),
]

train_dataloader = dict(
    batch_size=8,
    num_workers=5,
    dataset=dict(type='CustomDataset', data_root='images/train/', pipeline=train_pipeline),
)

optim_wrapper = dict(type='OptimWrapper', optimizer=dict(type='SGD', lr=0.01, momentum=0.9, weight_decay=0.0001))


data_preprocessor = dict(
    num_classes=10,
    # RGB format normalization parameters
    mean=[123.675, 116.28, 103.53],
    std=[58.395, 57.12, 57.375],
    # convert image from BGR to RGB
    to_rgb=True,
)

hook = dict(type='Hook', h=66666666)

train_cfg = dict(type='Loop', 
                 max_epochs=10, 
                 val_begin=50, 
                 val_interval=20)

cfg = dict(type='Runner',model=model,
                optim_wrapper=optim_wrapper,
                custom_hooks=hook,
                train_dataloader=train_dataloader,
                train_cfg=train_cfg,
                data_preprocessor=data_preprocessor)

runner = RUNNERS.build(cfg)


runner.train()